<a href="https://colab.research.google.com/github/faisalaldwaish/Sentiment-Analysis-of-Restaurant-Reviews/blob/main/full_review_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers gradio





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 692.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing 

In [2]:
pip install fuzzywuzzy[speedup]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 43.7 MB/s eta 0:00:00


In [5]:
import pandas as pd

# Load the uploaded file
file_path = 'Restaurant_reviews.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
df.head(150)

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures,7514
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0,2447.0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0,NaN
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0,NaN
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0,NaN
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0,NaN
...,...,...,...,...,...,...,...,...
145,Paradise,Aijaz,Had a good time and best service...near place ...,5,"2 Reviews , 167 Followers",5/20/2019 20:51,0,NaN
146,Paradise,Prashant Kumar K,Taste of the biriyani is well defined with the...,5,"1 Review , 1 Follower",5/19/2019 20:02,0,NaN
147,Paradise,Saikrishna CS,Biryani taste is good. Service is also good. T...,5,"1 Review , 2 Followers",5/19/2019 17:59,0,NaN
148,Paradise,Amar Stark,I like the way the staff treats the customers....,5,"2 Reviews , 1 Follower",5/19/2019 14:11,0,NaN


In [6]:
import pandas as pd
from transformers import pipeline
import gradio as gr
import seaborn as sns
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz

# تحميل البيانات
reviews_df = pd.read_csv('Restaurant_reviews.csv')

# تحميل نموذج التصنيف العاطفي
sentiment_model = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:

# وظيفة لتصنيف المراجعة
def classify_review(user_review):
    try:
        if not user_review.strip():
            return "Please enter a valid review."

        best_match = None
        best_score = 0

        for _, row in reviews_df.iterrows():
            if pd.isna(row['Review']):
                continue

            score = fuzz.token_sort_ratio(user_review.lower(), str(row['Review']).lower())
            if score > best_score:
                best_score = score
                best_match = row

        if best_score > 80:
            rating = best_match['Rating']
            rating_based_classification = f"Positive review based on rating: {rating}" if int(rating) >= 4 else f"Negative review based on rating: {rating}"

            sentiment_result = sentiment_model(user_review)[0]
            sentiment = sentiment_result['label']
            confidence = sentiment_result['score']
            sentiment_based_classification = f"Model prediction: {sentiment} with confidence: {confidence:.2f}"

            return f"{rating_based_classification}\n{sentiment_based_classification}\nMatching Score: {best_score}%"
        else:
            return "Review not found in the dataset."

    except Exception as e:
        return f"An error occurred: {str(e)}"


In [8]:

# وظيفة لرسم توزيع التقييمات
def plot_rating_distribution():
    plt.figure(figsize=(8, 6))
    sns.countplot(x='Rating', data=reviews_df, order=[1, 2, 3, 4, 5])
    plt.title('Distribution of Ratings')
    plt.xlabel('Rating')
    plt.ylabel('Count')
    plt.tight_layout()
    return plt.gcf()



In [9]:
# إنشاء واجهات Gradio
review_interface = gr.Interface(
    fn=classify_review,
    inputs=gr.Textbox(lines=2, placeholder="Enter your review here", label="Reviews"),
    outputs="text",
    title="Review Classifier Based on Rating and Hugging Face Model",
    description="Enter a restaurant review. The system will classify it based on the dataset rating and use a sentiment analysis model."
)

plot_interface = gr.Interface(
    fn=plot_rating_distribution,
    inputs=[],
    outputs="plot",
    title="Rating Distribution",
    description="Shows the distribution of ratings in the dataset."
)

# دمج الواجهتين باستخدام تبويبات
tabbed_interface = gr.TabbedInterface([review_interface, plot_interface], ["Review Classifier", "Rating Distribution"])

# إطلاق الواجهات
tabbed_interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://89d5bb0bb4e5ab2385.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
